In [10]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import platform as pl
import plotly.express as px
import plotly.graph_objects as go


In [11]:
file = "logs_abdi_2.csv"
escavadeira = 24
caminhao = 25

sep = '\\' if pl.uname().system == 'Windows' else '/'
df = pd.read_csv(f"..{sep}dados_csv{sep}{file}", encoding='iso-8859-1')


# Fator de Eficiência
Obs: A quantidade reduzida de aferições é resultante das limitações e inconsistências da comunicação serial entre o sistema embarcado e a balança Marpress da escavadeira a qual realiza a aferição do peso. Ademais, as amostragens não periódicas se devem a falta de operação do equipamento uma vez que necessita do condutor para ligá-lo ou desligá-lo.    

In [4]:
df_data = df.loc[(df['imei'] == escavadeira)]
df_data.loc[:,'date_created'] = pd.to_datetime(df_data['date_created'])
df_data['last_date'] = df_data['date_created'].shift(1)
df_data['interval'] = (df_data['date_created'] - df_data['last_date']) / np.timedelta64(1, 'm')
df_data = df_data.set_index('date_created')
df_data = df_data.loc['2021-08-06':]

df_total = df_data.copy()
df_data = df_data.loc[df_data['weight'] > 0]
# Agrupa por hora
df_data = df_data.groupby(pd.Grouper(freq='3600s')).sum()
df_total = df_total.groupby(pd.Grouper(freq='3600s')).max()

df_total['eficiencia'] = df_data['interval']
df_total['eficiencia'].fillna(0, inplace=True)
print(df_total.loc[df_total['eficiencia'] > 0, ['weight', 'eficiencia']])
fig = px.bar(df_total, y="eficiencia", title="Fator de Eficiencia")
fig.show()

                      weight  eficiencia
date_created                            
2021-08-06 12:00:00   9390.0    1.266667
2021-08-18 06:00:00  10250.0    5.766667


## Rendimento

In [5]:
df_rend = df_total.copy()
# Agrupa por dia
df_rend = df_rend.groupby(pd.Grouper(freq='86400s')).sum()
fig = px.bar(df_rend, y="weight", title="Rendimento")
fig.show()

## Aferição do tempo de carga

In [6]:
import plotly.graph_objects as go

df_temp = df_data.copy()
df_temp.rename(columns = {'interval':'load_time'}, inplace = True)
df_temp = df_temp.loc[df_temp['weight'] > 0, ['weight', 'load_time']]
df_temp['weight'] = df_temp['weight']/1000

print(df_temp)
fig = go.Figure()
fig.add_trace(go.Bar(x=df_temp.index,
                y=df_temp['load_time'],
                name='Tempo em Min',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=df_temp.index,
                y=df_temp['weight'],
                name='Carga em Ton.',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='Aferição do tempo de Carga',
    xaxis_tickfont_size=14,
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

                     weight  load_time
date_created                          
2021-08-06 12:00:00   93.26   1.266667
2021-08-18 06:00:00   41.30   5.766667


## Consumo de Combustível da Escavadeira

In [7]:
df_comb = df_total.copy()
litragem = 345
df_max = df_comb.groupby(pd.Grouper(freq='86400s')).max().fillna(0)
df_min = df_comb.groupby(pd.Grouper(freq='86400s')).min().fillna(0)
df_comb = df_comb.groupby(pd.Grouper(freq='86400s')).max().fillna(0)
diff_fuel = df_max['fuel'] - df_min['fuel']
df_comb['diff_fuel'] = diff_fuel
df_comb['consumo'] = (diff_fuel*litragem)/100
# print(df_comb.loc[:, ['diff_fuel', 'consumo']])

fig = go.Figure()
fig.add_trace(go.Bar(x=df_comb.index,
                y=df_comb['diff_fuel'],
                name='Nivel Combustivel (%)',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=df_comb.index,
                y=df_comb['consumo'],
                name='Consumo em Litros',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='Consumo de Combustível da Escavadeira',
    xaxis_tickfont_size=14,
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

## Consumo de Combustível do Caminhão

In [8]:
df_data = df.loc[(df['imei'] == caminhao)]
df_data.loc[:,'date_created'] = pd.to_datetime(df_data['date_created'])
df_data = df_data.set_index('date_created')
df_data = df_data.loc['2021-08-06':]

df_comb = df_data.copy()
litragem = 300
df_max = df_comb.groupby(pd.Grouper(freq='86400s')).max().fillna(0)
df_min = df_comb.groupby(pd.Grouper(freq='86400s')).min().fillna(0)
df_comb = df_comb.groupby(pd.Grouper(freq='86400s')).max().fillna(0)
diff_fuel = df_max['fuel'] - df_min['fuel']
df_comb['diff_fuel'] = diff_fuel
df_comb['consumo'] = (diff_fuel*litragem)/100
# print(df_comb.loc[:, ['diff_fuel', 'consumo']])

fig = go.Figure()
fig.add_trace(go.Bar(x=df_comb.index,
                y=df_comb['diff_fuel'],
                name='Consumo de Combustivel (%)',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=df_comb.index,
                y=df_comb['consumo'],
                name='Consumo em Litros',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='Consumo de Combustível do Caminhão',
    xaxis_tickfont_size=14,
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

## Peso Transportado por Combustível Consumido

Obs: Uma vez que os caminhões não possuem sensores para aferição de carga transportada, este indicador se torna inviável de ser mensurado.

## Trajeto Ciclo de Carga

In [57]:
# Plota Gráfico
def plotDatas(fig, datas, is_line=True, label=''):
        if fig is None:
            fig = go.Figure(go.Scattermapbox(
            mode = "markers+lines" if is_line == True else "markers",
            lon = datas[:,1],
            lat = datas[:,0],
            text = label if label != '' else 'veic1',
            marker = {'size': 10,
                    'color': 'blue'}))
        else:
            fig.add_trace(go.Scattermapbox(
            mode = "markers+lines" if is_line == True else "markers",
            lon = datas[:,1],
            lat = datas[:,0],
            text = label if label != '' else 'veic2',
            marker = {'size': 10,
                    'color': 'red'}))    

        fig.update_layout(
            margin ={'l':0,'t':0,'b':0,'r':0},
            #'center': {'lon': -38.71901, 'lat':  -3.94467},
            mapbox = {
                'center': {'lon': datas[:,1].mean(), 'lat':  datas[:,0].mean()},
                'style': "stamen-terrain",
                'center': {'lon': datas[:,1].mean(), 'lat':  datas[:,0].mean()},
                'zoom': 16})
        return fig

In [68]:
df_end = df.loc[(df['imei'] == caminhao) & (df['status'] == 'Descarregando')]
df_end['date_created'] = pd.to_datetime(df_end['date_created'])
print(df_end.iloc[0])
df_init = df.loc[(df['imei'] == escavadeira) & (df['status'] == 'Carregando')]
df_init['date_created'] = pd.to_datetime(df_init['date_created'])
df_init = df_init.loc[(df['date_created'] >= '2021-08-06') & (df['date_created'] < '2021-08-07')]
print(df_init.iloc[-1])
df_interval = df.loc[df['imei'] == caminhao]
df_interval['date_created'] = pd.to_datetime(df_interval['date_created'])
df_interval = df_interval.loc[(df_interval['date_created'] >= '2021-08-06 12:52:59') & (df_interval['date_created'] <= '2021-08-06 13:37:00')]
df_end


latitude                  -3.935874
longitude                 -38.72076
status                Descarregando
angle                    -10.112114
fuel                          106.0
weight                         -1.0
velocity                   2.164009
imei                             25
date_created    2021-08-06 13:37:00
Name: 1321, dtype: object
latitude                  -3.935989
longitude                -38.720802
status                   Carregando
angle                     50.139523
fuel                           89.0
weight                       9390.0
velocity                   0.539458
imei                             24
date_created    2021-08-06 12:52:59
Name: 596, dtype: object


,latitude,longitude,status,angle,fuel,weight,velocity,imei,date_created
1321,-3.935874,-38.720760,Descarregando,-10.112114,106.0,-1.0,2.164009,25,2021-08-06 13:37:00
1323,-3.935922,-38.720768,Descarregando,-6.814188,102.0,-1.0,1.290993,25,2021-08-06 13:37:06
1325,-3.935941,-38.720776,Descarregando,-9.963890,101.0,-1.0,0.413859,25,2021-08-06 13:37:11
1328,-3.935962,-38.720783,Descarregando,-6.614802,100.0,-1.0,0.070006,25,2021-08-06 13:37:21
1330,-3.935963,-38.720779,Descarregando,-5.981646,100.0,-1.0,0.339735,25,2021-08-06 13:37:27
...,...,...,...,...,...,...,...,...,...
7773,-3.931060,-38.724079,Descarregando,0.000000,86.0,-1.0,0.341794,25,2021-08-07 14:56:00
7774,-3.931061,-38.724083,Descarregando,0.000000,86.0,-1.0,0.586815,25,2021-08-07 14:56:16
11358,-3.942091,-38.707874,Descarregando,0.000000,80.0,-1.0,0.555930,25,2021-08-11 07:18:42
11424,-3.924900,-38.730087,Descarregando,0.000000,88.0,-1.0,0.611523,25,2021-08-11 10:25:15


In [63]:
gps_vals = df_interval.loc[:,['latitude','longitude']]
fig1 = None
fig1 = plotDatas(fig1, gps_vals.values, is_line=True, label='caminhao')
fig1.show()